# CSMCC16 Flight Coursework

## Set up

### Import packages

In [1]:
import pandas as pd
import datetime
from difflib import SequenceMatcher
import PySimpleGUI as sg
import math

### Create Functions

In [2]:
#calculate datetime values from epoch and arrival time
def calc_arrive_time(df):
    df['Arrive_epoch'] = pass_data['Depart_time_epoch']+df['Flight_time_min']*60
    df['Depart_time_datetime'] = pd.to_datetime(df['Depart_time_epoch'],unit='s')
    df['Arrive_time_datetime'] = pd.to_datetime(df['Arrive_epoch'],unit='s')
    df['Depart_time']=df['Depart_time_datetime'].dt.strftime("%H:%M:%S")

In [3]:
#create route (to use as unique id)
def route(df):
    df['Route'] = df['From_airport_IATA']+'-' + df['Dest_airport_IATA']+'-' + df['Depart_time']

In [4]:
#break route for details
def break_route(df):
    df[['From_airport_IATA','Dest_airport_IATA','Depart_time']]=df['Route'].str.split('-',expand=True)

In [5]:
# Remove any rows that do not fit good_format
def remove_bad_format(df,column,good_format):
    df =df.loc[df[column].str.contains(good_format)==True]
#add reset index?
    return df

In [6]:
# Remove row if not in a given list
def remove_list(df,column,list1):
    df = df[df[column].isin(list1)]
#     df.reset_index(inplace=True, drop = True)
    return df   

In [7]:
# Calculate number of passengers per flights
def pass_p_flight(df):
    df2 = df[['Flight_id','Passenger_id']]
    df_new = df2.groupby('Flight_id').count()
#     df_new.reset_index(inplace=True, drop = True)
    return df_new 

In [8]:
def remove_if_1(df,column):
    df = df.loc[df[column]!=1]
    df.reset_index(inplace=True)
    return df

In [13]:
# to view all 500 records, not top 5 and bottom 5
pd.set_option("display.max_rows", None, "display.max_columns", None)

### GUI settings

In [9]:
sg.SetOptions(
    background_color='#152238',
       text_element_background_color='#152238',
       element_background_color='#152238',
       text_color='#FFFFFF',
       scrollbar_color=None,
       input_elements_background_color='#FFFFFF',
       progress_meter_color = ('green', 'blue')
#        button_color=('white','#475841')
    )

True

## Import Data

In [10]:
#import csv as dataframe
pass_data = pd.DataFrame(pd.read_csv('AComp_Passenger_data.csv',header=None))
airport_lat_lon = pd.DataFrame(pd.read_csv('Top30_airports_LatLong.csv',header=None))

In [11]:
# Add Column Names
pass_data.columns = ['Passenger_id'
                        ,'Flight_id'
                        ,'From_airport_IATA'
                        ,'Dest_airport_IATA'
                        ,'Depart_time_epoch'
                        ,'Flight_time_min']
airport_lat_lon.columns = ['Airport_name'
                        ,'Airport_IATA_code'
                        ,'Latitude'
                        ,'Longitude']

### Add extra columns

In [14]:
#Add flight time info 
calc_arrive_time(pass_data)

## Error Correction

### Remove missing values

In [12]:
#Remove values where passenger id and flight id are missing
pass_data=pass_data.dropna(subset=['Flight_id','Passenger_id'])

### Flight data only

In [15]:
#select subset of data 
flight_info = ['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time'         
                        ]
flights = pass_data[flight_info]
#Group data by flight
flights = flights.groupby(flight_info).count()
#reset index
flights.reset_index(inplace=True)
#add route info
route(flights)
# flights

In [16]:
#Remove flights that do not fit the airport format
format_IATA = r"[A-Z][A-Z][A-Z]"
flights=remove_bad_format(flights,'From_airport_IATA',format_IATA)
flights=remove_bad_format(flights,'Dest_airport_IATA',format_IATA)

In [17]:
#Remove flights that do not fit the flight id format
format_flightid = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z]"
flights=remove_bad_format(flights,'Flight_id',format_flightid)

In [18]:
# Remove rows where airport is not in airport list
flights = remove_list(flights
                      ,'From_airport_IATA'
                      ,airport_lat_lon['Airport_IATA_code'
                                      ])
flights = remove_list(flights
                      ,'Dest_airport_IATA'
                      ,airport_lat_lon['Airport_IATA_code'
                                      ])

In [19]:
# Calculate flights per passenger
flights_pass = pass_p_flight(pass_data)
# Remove if just 1 flight
flights_pp = remove_if_1(flights_pass,'Passenger_id')
# flights_pp

In [20]:
# Final flight list, only keep flights in the flight id list
flights = remove_list(flights,'Flight_id',flights_pp['Flight_id'])

In [21]:
flights.reset_index(inplace=True, drop = True)

In [22]:
# flights

## add to passenger data

In [23]:
#Add route info to pass_data
route(pass_data)
# pass_data

In [24]:
#Select data required
pass_data_slim = pass_data[['Passenger_id','Flight_id','Route']]
# pass_data_slim

In [25]:
# correct flight info
def correct_flight_err (df,column1,column2):
    for ind in df.index:
        if (df[column1][ind] in flights[column1].to_list()) == False:
            #  if column 1 is wrong, use column 2
            use_column2 = df[column2][ind]
            #  get the row number of correct value
            row = flights[flights[column2] == use_column2].index[0]
            #  take the correct value from flights table
            df.loc[ind,column1]=flights[column1][row]
    return df

In [26]:
# pass_data_slim

In [27]:
def pass_p_flight(df):
    df2 = pass_data_slim[['Flight_id','Passenger_id']]
    df_new = df2.groupby('Flight_id').count()
#     df_new.reset_index(inplace=True)
    return df_new 

In [28]:
pass_data_slim = correct_flight_err(pass_data_slim,'Route','Flight_id')
pass_data_slim = correct_flight_err(pass_data_slim,'Flight_id','Route')

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


## Error Correction
### Passenger

In [29]:
#Remove flights that do not fit the airport format
format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
# pass_data_route=
freq_fliers = remove_bad_format(pass_data_slim,'Passenger_id',format_pass_id)
# freq_fliers

In [30]:
def flight_per_pass(df):
    df2 = df[['Flight_id','Passenger_id']]
    df_new = df2.groupby('Passenger_id').count()
    df_new.reset_index(inplace=True)
    return df_new 

In [31]:
freq_fliers_pf=flight_per_pass(freq_fliers)
# freq_fliers_pf

In [32]:
#remove poor format
format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
freq_fliers_pf=remove_bad_format(freq_fliers_pf,'Passenger_id',format_pass_id)

In [33]:
freq_fliers_pf = remove_if_1(freq_fliers_pf,'Flight_id')
# freq_fliers_pf

In [34]:
# Update passenger id bsed on a checklist

def update_pass_id(df,check_list):
        true = check_list.to_list()
        for ind in pass_data_slim.index:
            f = df['Passenger_id'][ind]
            # if the vlaue is not in the checklist...
            if (f in true)==False:
                test_best_true = []
                final_true = []
                # Check against all true values
                for t in true:
                    # SequesnceMatch Ratio will provide a similarity score
                        test_best_true.append([t,round(SequenceMatcher(None, t, f).ratio(),3)])
                test_best_true = pd.DataFrame(test_best_true)
                # Replace the false value with the true value that has highest score
#                 df['Passenger_id'][ind]=test_best_true.iloc[test_best_true[1].idxmax(), 0]
                df.loc[ind,'Passenger_id']=test_best_true.iloc[test_best_true[1].idxmax(), 0]
#                 data.loc[data.bidder == 'parakeet2004', 'bidderrate'] = 100

        return df

In [35]:
pass_data_slim = update_pass_id(pass_data_slim,freq_fliers_pf['Passenger_id'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [36]:
pass_data_slim.reset_index(inplace=True, drop = True)
# pass_data_slim

## Objective 1: 
### Determine the number of flights from each airport; include a list of any airports not used.

In [37]:
break_route(pass_data_slim)
from_list = pass_data_slim['From_airport_IATA'].to_list()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [38]:
#mapper, where key = airport and value = 1 for each airport in from_list
key2val2 = [[a, 1] for a in from_list]
mapper = pd.DataFrame(key2val2, columns=['Airport_IATA_code','value'])
#mapper

#reducer - takes key and val from mapper to provide final key and value
reducer=mapper.groupby(['Airport_IATA_code']).sum()
reducer
reducer.reset_index(inplace=True)

In [39]:
# merge full list of airports with number of flights from each airport
airport_list = pd.DataFrame(airport_lat_lon['Airport_IATA_code'])
# airport_list=airport_list)
num_flights = airport_list.merge(reducer, how='outer', on=['Airport_IATA_code'])

In [40]:
# clean up results
num_flights=num_flights.fillna(0)
num_flights=num_flights.sort_values(by=['value','Airport_IATA_code'], ascending=[False,True])
# not sure whether to do the one below.... i think it looks neater....
num_flights.set_index('Airport_IATA_code', inplace=True, drop=True)
num_flights

,value
Airport_IATA_code,
DEN,46.0
CAN,37.0
IAH,37.0
ATL,36.0
KUL,33.0
ORD,33.0
CGK,27.0
JFK,25.0
LHR,25.0


## Objective 2
### Create a list of flights based on the Flight id, this output should include the passenger Id, relevant IATA/FAA codes, the departure time, the arrival time (times to be converted to HH:MM:SS format), and the flight times.

## Objective 3
### Calculate the number of passengers on each flight.

In [41]:
# pass_data_route
# calc_arrive_time(pass_data_route)
#num_flights = airport_list.merge(from_counts_df, how='outer', on=['Airport_IATA_code'])

In [42]:
flights = pass_data_slim['Flight_id']

In [43]:
#mapper, where key = airport and value = 1 for each airport in from_list
key2val2 = [[a, 1] for a in flights]
mapper = pd.DataFrame(key2val2, columns=['Flight_id','value'])
#mapper

#reducer - takes key and val from mapper to provide final key and value
num_passengers=mapper.groupby(['Flight_id']).sum()
#num_passengers
#reducer
num_passengers.reset_index(inplace=True)

In [44]:
# clean up results
#num_passengers=num_flights.fillna(0)
num_passengers=num_passengers.sort_values(by=['value','Flight_id'], ascending=[False,True])
num_passengers.set_index('Flight_id', inplace=True, drop=True)
num_passengers

,value
Flight_id,
ULZ8130D,27
GMO5938W,25
XXQ4064B,25
KJR6646J,23
QHU1140O,21
SQU6245R,21
WSK1289Z,21
FYL5866L,20
JVY9791G,20


## Objective 4
### Calculate the line-of-sight (nautical) miles for each flight and the total travelled by each passenger and thus output the passenger having earned the highest air miles.

In [45]:
# Calulate miles per flight,

def calculate_miles(df_in):
    global miles_per_flight
    miles_per_flight = []
    for ind in df_in.index:
            #Take from airport
            p= df_in['Passenger_id'][ind]
            x= df_in['From_airport_IATA'][ind]
            #Find it in airport lat long list
            row = (airport_lat_lon[airport_lat_lon['Airport_IATA_code'] == x].index[0])
            #Give me the latitude and longitude
            a = airport_lat_lon['Latitude'][row]
            b = airport_lat_lon['Longitude'][row]
            #Take dest airport
            y = df_in['Dest_airport_IATA'][ind]
            #Find it in airport lat long list
            row = (airport_lat_lon[airport_lat_lon['Airport_IATA_code'] == y].index[0])
            #Give me the latitude and longitude
            c = airport_lat_lon['Latitude'][row]
            d = airport_lat_lon['Longitude'][row]
            miles = math.sqrt((c-a)**2+(d-b)**2)
            miles_per_flight.append([df_in['Passenger_id'][ind],miles])
    miles_per_flight= pd.DataFrame(miles_per_flight
                                   , columns=['Passenger_id','Miles']
                                  )    


In [46]:
calculate_miles(pass_data_slim)
# miles_per_flight

In [47]:
key2val2 = pd.DataFrame()
for index,row in miles_per_flight.iterrows():
    key2val2=key2val2.append(miles_per_flight.loc[index])
    mapper = pd.DataFrame(key2val2, columns=['Miles','Passenger_id'])
# mapper

In [48]:
#reducer - takes key and val from mapper to provide final key and value
miles_per_pass=mapper.groupby(['Passenger_id']).sum().round(1)
#reducer
miles_per_pass.reset_index(inplace=True)

In [49]:
# clean up results
#num_passengers=num_flights.fillna(0)
miles_per_pass=miles_per_pass.sort_values(by=['Miles'], ascending=[False])
miles_per_pass.set_index('Passenger_id', inplace=True, drop=True)
miles_per_pass

,Miles
Passenger_id,
UES9151GS5,3492.2
DAZ3029XA0,3305.9
BWI0520BG6,3272.9
SPR4484HA6,3128.0
PUD8209OG3,3096.7
WYU2010YH8,2976.8
JJM4724RF7,2686.0
WBE6935NU3,2600.8
CKZ3132BR4,2559.8
